In [1]:

import os
import sys
import numpy as np
import json as json
import pandas as pd
from datetime import datetime

# General Paths
data_path = os.getcwd()
project_path = os.path.dirname(data_path)
benchmarks_path = os.path.join(project_path, 'benchmarks')

SKAB_path = os.path.join(benchmarks_path, 'SKAB')
SKAB_dataset = os.path.join(SKAB_path, 'data')
SKAB_labels = os.path.join(SKAB_path, 'labels')

SKAB_clean_path = os.path.join(data_path, 'SKAB')


In [2]:
# Load the Task Manifest
task_manifest = {
    "anomaly-free": {
        "dataset": "SKAB",
        "name": "anomaly-free",
        "anomaly_keys": []
    },
    "other": {
        "dataset": "SKAB",
        "name": "other",
        "anomaly_keys": ["anomaly", "changepoint"]
    },
    "valve1": {
        "dataset": "SKAB",
        "name": "valve1",
        "anomaly_keys": ["anomaly", "changepoint"]
    },
    "valve2": {
        "dataset": "SKAB",
        "name": "valve2",
        "anomaly_keys": ["anomaly", "changepoint"]
    }
}

In [3]:
# Datasets
datasets = {
    'SKAB': {
        'path': SKAB_dataset,
        # 'labels': get_nab_labels,
        'ts_name': 'datetime'
    },
}


In [4]:
data = []

anoms = []

# Loop through each of the individual Tasks in the Dataset
for task_family_number, (task_family_name, task_data) in enumerate(task_manifest.items()):
    
    # Make the Directory for the Task
    proml_task_family = os.path.join(SKAB_clean_path, task_family_name)
    if not os.path.exists(proml_task_family):
        os.makedirs(proml_task_family)

    # Loop through the Data Files in the Task Dataset
    task_path = os.path.join(datasets[task_data['dataset']]['path'], task_data['name'])
    for i, task_file in enumerate(os.listdir(task_path)):
        
        # Get the Name of the Data File for the given Family
        task_file_name, _ = task_file.split('.')

        task_file_path = os.path.join(task_path, task_file)
        task_df = pd.read_csv(task_file_path, sep=';')

        # Loop thrugh the Task Data to Create Frames Consistent Size
        splits_per_task = 20
        split_nu = 0
        split_size = 50
        split_start = 0
        split_end = split_size
        # (While we can still get 20(tracks)x50(samples)
        while split_start + (split_size*splits_per_task) < len(task_df): 

            # Make the Charachter Directory
            char_name = f"{task_file_name}_{split_nu}"
            char_path = os.path.join(proml_task_family, char_name)
            if not os.path.exists(char_path):
                os.makedirs(char_path)

            # Make Sure there are 20 Samples
            group_samples = []
            for j in range(splits_per_task):

                # DO THE SPLITZ
                split_df = task_df.iloc[split_start:split_end, :]
                split_start += split_size
                split_end += split_size

                # Check to See if there is an ANOM in the SPLIT
                is_anom = False
                for anomaly_key in task_data['anomaly_keys']:
                    split_anomaly_labels = split_df[anomaly_key].values

                    if any([True if i == 1 else False for i in split_anomaly_labels ]):
                        is_anom = True

                # Remove the TS / Drop Columns for Anomaly Labels
                cols_to_drop = ['datetime']
                cols_to_drop.extend(task_data['anomaly_keys'])
                split_df = split_df.drop(labels=cols_to_drop, axis=1)

                # Save the Dataframe to a new ProML folder
                proml_file_path = os.path.join(char_path, f'{j+1}.csv')
                split_df.to_csv(proml_file_path, index=False)

                # Add the Sample to Tensor to be Loaded by VERSA
                split_arr = split_df.values.tolist()
                group_samples.append((np.array(split_arr), int(is_anom)))
                anoms.append(int(is_anom))
            
            # Append Data
            data.append(group_samples)

            # Increment Split NU
            split_nu += 1

data = np.array(data)
print(data.shape)
np.save('skab.npy', data)


(38, 20, 2)


<ipython-input-4-36e8c673d5fe>:75: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


In [5]:
sum(anoms)/len(anoms)

0.32894736842105265